In [ ]:
import pandas as pd
import numpy as np
from snappy import *


In [3]:
#load data

#knotinfo = pd.read_excel('/Users/seohyeonlee/knotinfo/data/knotinfo_data_complete.xls')
linkinfo = pd.read_excel('/Users/seohyeonlee/knotinfo/data/linkinfo_data_complete.xls')


In [4]:
linkinfo_glossary = linkinfo.iloc[0, :].copy()
linkinfo_glossary['determinant']

linkinfo_glossary.shape


(89,)

In [5]:
linkinfo.shape
linkinfo.describe()
linkinfo.isna().any()
linkinfo.head()

#remove first row
linkinfo = linkinfo.iloc[1:, :].reset_index(drop=True)
linkinfo


,name,name_anon,diagram,diagram_anon,name_unoriented,name_unoriented_anon,category,category_anon,knot_atlas,knot_atlas_anon,...,determinant_anon,signature,signature_anon,nullity,nullity_anon,unlinking_number,unlinking_number_anon,Unnamed: 86,weak_splitting_number,weak_splitting_number_anon
0,L2a1{0},diagram_display.php?L2a1{0},linkL2a1{0}-50.png,diagram_display.php?L2a1{0},L2a1,NaN,2,NaN,L2a1,http://katlas.math.toronto.edu/wiki/L2a1,...,NaN,1,NaN,0,NaN,1,NaN,NaN,1,NaN
1,L2a1{1},diagram_display.php?L2a1{1},linkL2a1{1}-50.png,diagram_display.php?L2a1{1},L2a1,NaN,2,NaN,L2a1,http://katlas.math.toronto.edu/wiki/L2a1,...,NaN,-1,NaN,0,NaN,1,NaN,NaN,1,NaN
2,L4a1{0},diagram_display.php?L4a1{0},linkL4a1{0}-50.png,diagram_display.php?L4a1{0},L4a1,NaN,4,NaN,L4a1,http://katlas.math.toronto.edu/wiki/L4a1,...,NaN,1,NaN,0,NaN,2,NaN,NaN,2,NaN
3,L4a1{1},diagram_display.php?L4a1{1},linkL4a1{1}-50.png,diagram_display.php?L4a1{1},L4a1,NaN,4,NaN,L4a1,http://katlas.math.toronto.edu/wiki/L4a1,...,NaN,-3,NaN,0,NaN,2,NaN,NaN,2,NaN
4,L5a1{0},diagram_display.php?L5a1{0},linkL5a1{0}-50.png,diagram_display.php?L5a1{0},L5a1,NaN,5,NaN,L5a1,http://katlas.math.toronto.edu/wiki/L5a1,...,NaN,1,NaN,0,NaN,1,NaN,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#4000+ knots missing name
linkinfo['name'].value_counts()

type(linkinfo['name'][8373])

noname = linkinfo.loc[linkinfo['name'].isna()]
noname.isna().sum()


name                          4187
name_anon                     4187
diagram                       4187
diagram_anon                  4187
name_unoriented               4187
                              ... 
unlinking_number              4187
unlinking_number_anon         4187
Unnamed: 86                   4187
weak_splitting_number         4187
weak_splitting_number_anon    4187
Length: 89, dtype: int64

In [7]:
#remove rows where all values are NaN. do the same with columns
linkinfo = linkinfo.dropna(axis=0, how='all')
linkinfo = linkinfo.dropna(axis=1, how='all')


In [9]:
import  re

#process linking matrices
#'matrix_zero' is true if the linking matrix consists of only 0s, false if not, NA if data is missing

def parse_matrix(row):
	if not pd.isna(row):
		nums = re.findall(r'-?\d+', row)
		#print(nums)
		#print(all(num == 0 for num in nums))
		all_zero = all(int(num) == 0 for num in nums)
		#row = row.replace('{', '[').replace('}', ']')
	else:
		all_zero = pd.NA
	return all_zero

linkinfo['matrix_zero'] = linkinfo['linking_matrix'].apply(lambda x: parse_matrix(x))


In [12]:
determinant_is_zero = linkinfo['determinant'].apply(lambda row: row == 0)

#158 links whose determinant is zero
linkinfo['determinant_is_zero'] = determinant_is_zero
linkinfo['determinant_is_zero']


0       False
1       False
2       False
3       False
4       False
        ...  
4183     True
4184     True
4185     True
4186     True
4187     True
Name: determinant_is_zero, Length: 4188, dtype: bool

In [13]:
import math
#test if the determinant is a perfect square
determinant_is_square = linkinfo['determinant'].apply(lambda row: math.sqrt(row) % 1 == 0)
determinant_is_square

linkinfo['determinant_is_square'] = determinant_is_square


In [14]:
linkinfo.columns


Index(['name', 'name_anon', 'diagram', 'diagram_anon', 'name_unoriented',
       'category', 'knot_atlas', 'knot_atlas_anon', 'alternating',
       'orientation', 'name_rank', 'unoriented_name_rank', 'gauss_notation',
       'pd_notation_vector', 'pd_notation_math', 'crossing_number',
       'braid_index', 'braid_notation', 'braid_notation_old',
       'quasi_positive_braid', 'multivariable_alexander', 'conway_polynomial',
       'jones_polynomial', 'jones_polynomial_vector', 'homflypt_polynomial',
       'homflypt_polynomial_vector', 'kauffman_polynomial',
       'kauffman_polynomial_vector', 'khovanov_polynomial',
       'khovanov_polynomial_vector', 'unoriented', 'arc_notation',
       'linking_matrix', 'rolfsen_name', 'volume', 'components', 'dt_code',
       'arf_invariant', 'smooth_four_genus', 'topological_four_genus',
       'seifert_matrix', 'splitting_number', 'determinant', 'signature',
       'nullity', 'unlinking_number', 'weak_splitting_number', 'matrix_zero',
       'det

In [15]:

#aggregate information needed to check whether link is ribbon (add more properties later)
ribbon_info = linkinfo[['name_unoriented', 'linking_matrix', 'components', 'determinant', 'matrix_zero', 'determinant_is_zero', 'smooth_four_genus']]
ribbon_info = ribbon_info.drop_duplicates(keep='first')

matrix_and_determinant_both_zero = ribbon_info['matrix_zero'] * ribbon_info['determinant_is_zero']

ribbon_info['matrix_and_determinant_both_zero'] = matrix_and_determinant_both_zero

matrix_and_determinant_both_zero.value_counts()

ribbon_info[matrix_and_determinant_both_zero].name_unoriented.tolist()


['L9n27',
 'L10n32',
 'L10n36',
 'L10n56',
 'L10n57',
 'L10n59',
 'L10n107',
 'L11n218',
 'L11n226',
 'L11n244',
 'L11n247',
 'L11n381',
 'L11n396',
 'L11n404',
 'L11n406',
 'L11n436']

In [16]:
#knots whose linking matrix are zero
ribbon_info[ribbon_info['matrix_zero']]
#knots whose determinant (calculated by plugging -1 into the alexander polynomial) are zero
ribbon_info[ribbon_info['determinant_is_zero']]

cleaned_linkinfo = ribbon_info[ribbon_info['matrix_zero'] & ribbon_info['determinant_is_zero']]

cleaned_linkinfo.shape


(16, 8)

In [17]:
#export the version of the linkinfo table with empty values removed and links with nonzero linking matrices identified in a separate column
cleaned_linkinfo.to_csv('results/cleaned_linkinfo.csv', index=False)


In [18]:
#takes roughly a minute to run on 3000+ links
#runtime cut to 6.3s after cutting down on to 16 knots to check
!sage "calculate_jones.sage"


sage script running


In [19]:
sage_output = pd.read_csv('/Users/seohyeonlee/knotinfo/results/sage_output.csv')


In [20]:
sage_output.shape


(16, 11)

In [ ]:
#16 links whose determinant is 0 and linking matrix has only zeroes
#out[(out['determinant'] == 0) & out['matrix_zero'] ]

#filtered links that satisfy 2 conditions (determinant 0, matrix only 0) and whose jones determinants are defined
potential_ribbons = sage_output[(sage_output['determinant'] == 0) & sage_output['matrix_zero'] & sage_output['jones_factorable'] & (sage_output['smooth_four_genus'] == '0')]
potential_ribbons.to_csv('/Users/seohyeonlee/knotinfo/results/potential_ribbons_upto11_crossings.csv', index=False)


NameError: name 'potential_ribbons_upto11' is not defined